In [68]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import string

## Strategies to try out
*Say we have a budget of $10 per game*

### Baselines :
- Random
- Bet $1 on each player
- Bet $2 on each player on home team

### With model :
- Stake formula
- Optimization -- balance expected returns and variance
- Weight by jumpball probability

In [42]:
def normalize_name(x) :

    for suffix in [' Jr.', ' Sr.', ' III', ' II', ' IV', ' Jr', ' Sr'] :
        x = x.replace(suffix, '')
    x = x.translate(str.maketrans('', '', string.punctuation))

    return unidecode(x).lower()

In [95]:
odds = pd.read_csv('../data/odds_first_basket.csv')
odds = odds[
    odds
    .groupby(['name', 'event_id', 'bookmaker'])
    ['update_time'].transform('max') == odds['update_time']
].reset_index(drop = True)
odds = odds[odds['bookmaker'] == 'fanduel']
first_basket = pd.read_csv('../data/first_basket_2025.csv')
metadata = pd.read_csv('../data/player_metadata.csv')

In [98]:
odds['prob'] = 1 / odds['price']

odds.groupby('game_id')['prob'].sum()

game_id
202412010BRK    1.152708
202412030DAL    1.166622
202412030DEN    1.147607
202412030LAC    1.163408
202412030NYK    1.087681
202412030SAC    1.179121
Name: prob, dtype: float64

In [92]:
odds = odds.merge(
    first_basket[['game_id', 'first_basket']],
    on = 'game_id',
    how = 'left'
)

odds = odds.merge(
    metadata[['player_id', 'name']],
    left_on = 'first_basket', right_on = 'player_id',
    how = 'left'
)

odds['name_x'] = odds['name_x'].apply(normalize_name)
odds['name_y'] = odds['name_y'].apply(normalize_name)

odds['first_basket_scorer'] = (odds['name_x'] == odds['name_y']).astype(int)

In [93]:
def evaluate_strategy(odds, strategy = 'random') :

    if strategy == 'random' :

        odds['stake'] = np.random.uniform(0, 2, (odds.shape[0],))

    odds['return'] = odds['stake'] * (odds['price'] * odds['first_basket_scorer'] - 1)

    return odds

In [94]:
evaluate_strategy(odds).sort_values('return', ascending = False)

,name_x,price,bookmaker,update_time,game_id,event_id,insert_timestamp_utc,first_basket,player_id,name_y,first_basket_scorer,stake,return
9,goga bitadze,11.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,1,1.171144,11.711443
53,norman powell,8.0,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,1,1.522959,10.660714
31,michael porter,8.0,fanduel,2024-12-04T02:45:15Z,202412030DEN,eaf04a3044de875dc6c83132d04ae9f6,2024-12-04 02:45:16.563756+00:00,portemi01,portemi01,michael porter,1,1.514925,10.604477
39,alperen sengun,6.0,fanduel,2024-12-04T02:44:25Z,202412030SAC,30f75d85c74e0811f06c0fbe86044f54,2024-12-04 02:45:16.619025+00:00,sengual01,sengual01,alperen sengun,1,1.812952,9.064758
19,jaren jackson,5.7,fanduel,2024-12-04T01:13:58Z,202412030DAL,6060ee0918c66681cee75352d305f91c,2024-12-04 01:15:41.431451+00:00,jacksja02,jacksja02,jaren jackson,1,1.003491,4.716407
10,franz wagner,6.0,fanduel,2024-12-04T00:16:13Z,202412030NYK,823e03f1e150f589e94392112743e204,2024-12-04 00:16:23.639726+00:00,wagnefr01,wagnefr01,franz wagner,1,0.285811,1.429056
50,ivica zubac,7.0,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,0,0.025747,-0.025747
3,jalen wilson,9.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0,0.027103,-0.027103
23,kyrie irving,8.0,fanduel,2024-12-04T01:13:58Z,202412030DAL,6060ee0918c66681cee75352d305f91c,2024-12-04 01:15:41.431451+00:00,jacksja02,jacksja02,jaren jackson,0,0.088706,-0.088706
6,ben simmons,10.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0,0.097782,-0.097782


In [ ]:
odds.groupby()

,name_x,price,bookmaker,update_time,game_id,event_id,insert_timestamp_utc,first_basket,player_id,name_y,stake,first_basket_scorer,return
0,franz wagner,5.6,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0.127756,0,-0.127756
1,jalen suggs,7.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0.256646,0,-0.256646
2,dennis schroder,8.5,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0.709254,0,-0.709254
3,jalen wilson,9.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0.056319,0,-0.056319
4,cameron johnson,9.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,0.910452,0,-0.910452
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,jerami grant,8.5,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,1.851844,0,-1.851844
65,shaedon sharpe,8.5,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,0.684682,0,-0.684682
66,toumani camara,12.0,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,0.192829,0,-0.192829
67,derrick jones,13.0,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,0.423291,0,-0.423291


In [79]:
odds = pd.read_csv('../data/odds_first_basket.csv')

In [90]:
odds = odds[
    odds
    .groupby(['name', 'event_id', 'bookmaker'])
    ['update_time'].transform('max') == odds['update_time']
].reset_index(drop = True)

In [84]:
odds

,name,price,bookmaker,update_time,game_id,event_id,insert_timestamp_utc
0,Franz Wagner,5.6,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00
1,Jalen Suggs,7.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00
2,Dennis Schroder,8.5,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00
3,Jalen Wilson,9.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00
4,Cameron Johnson,9.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00
...,...,...,...,...,...,...,...
274,Shaedon Sharpe,8.5,bovada,2024-12-04T03:29:31Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00
275,Norman Powell,9.0,bovada,2024-12-04T03:29:31Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00
276,Derrick Jones,13.0,bovada,2024-12-04T03:29:31Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00
277,Toumani Camara,14.0,bovada,2024-12-04T03:29:31Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00
